In [ ]:
!pip install "pyautogen[retrievechat]==0.2.6" -qqq

# **OpenAI API 키 설정**

In [ ]:
import json

openai_api_key = "자신의 API 키 입력"

with open('OAI_CONFIG_LIST.json', 'w') as f:
  config_list = [
      {
          "model": "gpt-4-turbo-preview",
          "api_key": openai_api_key
      },
      {
          "model": "gpt-4o",
          "api_key": openai_api_key,
      },
      {
          "model": "dall-e-3",
          "api_key": openai_api_key,
      }
  ]

  json.dump(config_list, f)

# **에이전트에 사용할 설정 불러오기**

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    file_location=".",
    # 특정 모델만 사용하도록 필터링
    filter_dict={
        "model": {"gpt-4-turbo-preview"},
    },
)

llm_config = {
    "config_list": config_list,
    "temperature": 0,
}

# **AutoGen의 핵심 구성요소인 UserProxyAgent와 AssistantAgent**

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# AI 모델을 실행하는 역할
# AssistantAgent("에이전트 이름", AI 모델 설정)
assistant = AssistantAgent("assistant", llm_config=llm_config)
# 사용자를 대신해 명령을 내리는 에이전트
# 터미널 명령을 실행하거나 AI가 생성한 코드를 실행할 수도 있다
user_proxy = UserProxyAgent("user_proxy", # 에이전트 이름
                            # 종료 조건 설정
                            # 현재 조건은 content 값이 있는지 확인하고
                            # rstrip() -> 오른쪽 공백 제거
                            # 문자열이 TERMINATE로 끝나면 종료
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                            # 사용자가 직접 입력할 수 없도록 설정
                            # "ALWAYS"로 설정하면 AI가 사용자에게 입력을 요청할 수도 있음
                            human_input_mode="NEVER",
                            # "work_dir": "coding" -> 코드를 실행할 작업 폴더
                            # "user_docker": False 도커환경에서 실행하지 않도록 설정
                            code_execution_config={"work_dir": "coding", "user_docker": False})

# **삼성전자의 3개월 주식 가격 그래프를 그리는 작업 실행**

In [ ]:
# 에이전트와 대화를 시작
user_proxy.initiate_chat(assistant, message="""
삼성전자의 지난 3개월 주식 가격 그래프를 그려서 samsung_stock_price.png 파일로 저장해줘.
plotly 라이브러리를 사용하고 그래프 아래를 투명한 녹색으로 채워줘.
값을 잘 확인할 수 있도록 y축은 구간 최소값에서 시작하도록 해줘.
이미지 비율은 보기 좋게 적절히 설정해줘.
""")